In [11]:
import numpy as np
from scipy.io import loadmat

import matplotlib.pyplot as plt
%matplotlib inline 

## Let's start from linear SVM 
from sklearn.svm import LinearSVC
# http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
from sklearn import svm

## load one epoch data

In [2]:
def load_one_epoch(subject, data_type, epoch_num):
    if data_type is 'train':
        epoch_data=loadmat("data/{}{}-allfilt20.mat".format(subject, epoch_num))

    x=epoch_data['x']
    y=epoch_data['y']

    return x, y

In [3]:
subject='A'
epoch_num = 1
x, y = load_one_epoch(subject, 'train', 1)

print(x.shape)
print(y.shape)

## load all training data for a subject

In [4]:
def load_data(subject, data_type, num_epoches):
    X = list()
    Y = list()
    for epoch_num in range(num_epoches):
        #print(epoch_num)
        x, y = load_one_epoch(subject, data_type, 1+epoch_num)

        X.append(x)
        Y.append(y)


    X = np.array(X)
    Y = np.array(Y)

    print('loaded:')
    print(X.shape)
    print(Y.shape)

    num_trials = X.shape[1]
    data_dim = X.shape[2]

    # stack epoches
    X = X.reshape(-1,data_dim)
    Y = Y.reshape(-1,1)

    print('stacked:')
    print(X.shape)
    print(Y.shape)
    
    return X, Y

In [5]:

subject = 'A'
#subject = 'B'
num_epoches = 85

X_train, Y_train = load_data(subject, 'train', num_epoches)



## load and apply channel masks 

In [6]:
def load_channel_mask(subject, num_channels):
    return np.loadtxt('mask/{}_chosen_channel_mask_{}.txt'.format(subject,num_channels), dtype=int)

In [7]:
mask = load_channel_mask(subject,2)
#plt.plot(mask)
print(mask)

In [8]:
def apply_mask(X, mask):
    data_dim = X.shape[1]
    num_channels = len(mask)
    samples_per_channel = int(data_dim / num_channels)
    
#     print(X.shape)
#     print(data_dim)
#     print(num_channels)
#     print(samples_per_channel)    
    
    index = np.repeat(mask,samples_per_channel)
    
#     print(mask)
#     print(np.repeat(mask,samples_per_channel))

    return X[:,index>0]

In [15]:
X_train_chosen=apply_mask(X_train, mask)
X_train_chosen.shape

(15300, 28)

In [16]:
Y_train.shape

(15300, 1)

## Try training without scaling

In [18]:
# try training
clf = svm.LinearSVC(C=10, loss="hinge", max_iter=5000, random_state=42)  # class_weight='balanced',

clf.fit(X_train_chosen,Y_train.ravel())

/Users/yyoo/tf11/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=5000, multi_class='ovr',
     penalty='l2', random_state=42, tol=0.0001, verbose=0)

In [20]:
# training classfication error
Yhat = clf.predict(X_train_chosen)
print(np.mean(Y_train==Yhat))
print(np.std(Y_train==Yhat))

0.6947276688453159
0.46052267586522366


## Scaling matters!

In [24]:
## with scaling
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=10, loss="hinge", max_iter=5000, random_state=42)), # class_weight='balanced', 
    ])

clf.fit(X_train_chosen,Y_train.ravel())

/Users/yyoo/tf11/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=5000, multi_class='ovr',
     penalty='l2', random_state=42, tol=0.0001, verbose=0))])

In [25]:
# training classfication error
Yhat = clf.predict(X_train_chosen)
print(np.mean(Y_train==Yhat))
print(np.std(Y_train==Yhat))

## SCALING IS IMPORTANT!

0.8333333333333334
0.3726779962500039


## Load test data

In [ ]:
## TODO


**below is old code from https://github.com/ys7yoo/human-plus/blob/master/classifyUsingSVM.ipynb**

## Let's do cross-validation

In [ ]:
## cross validation
from sklearn.model_selection import cross_val_score
from sklearn import svm

# linear classifier
clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=10, loss="hinge",  random_state=42)), # class_weight='balanced',
    ])

# run cross validation
scores = cross_val_score(clf, X, Y, cv=5)

print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# repeat for a wide range of C
Cs = 10**np.linspace(-20,20,41)
Cs

k = 5
scores = list()
for C in Cs:
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=C, loss="hinge",  random_state=42)), # class_weight='balanced',
    ])

    scores.append(cross_val_score(clf, X, Y, cv=k))
    
#scores

mean_scores = np.mean(scores,axis=1)
#print(mean_scores)

maxIdx=np.argmax(mean_scores)
print("max accuracy = {} when C={}".format(mean_scores[maxIdx], Cs[maxIdx]))




import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(np.log10(Cs),mean_scores)
plt.plot(np.log10(Cs), baseAcc*np.ones_like(Cs), '--')
plt.xlabel('log_10(C)')
plt.ylabel('accuracy')
    

    


## Adding `balancing` 


In [ ]:
# repeat for a wide range of C
Cs = 10**np.linspace(-20,20,41)
Cs

k = 5
scores = list()
for C in Cs:
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=C, loss="hinge",  class_weight='balanced', random_state=42)),  # balanced!
    ])

    scores.append(cross_val_score(clf, X, Y, cv=k))
    
#scores

mean_scores_balanced = np.mean(scores,axis=1)
#print(mean_scores_balanced)

maxIdx=np.argmax(mean_scores_balanced)
print("max accuracy = {} when C={}".format(mean_scores_balanced[maxIdx], Cs[maxIdx]))




import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(np.log10(Cs),mean_scores)
plt.plot(np.log10(Cs),mean_scores_balanced)
plt.plot(np.log10(Cs), baseAcc*np.ones_like(Cs), '--')
plt.legend(('equal weight','balanced', 'baseline'))
plt.xlabel('log_10(C)')
plt.ylabel('accuracy')
    

In [ ]:
# let's repeat & check confusion matrix!

clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=10, loss="hinge", random_state=42)),  # balanced!
    #("linear_svc", LinearSVC(C=10, loss="hinge",  class_weight='balanced', random_state=42)),  # balanced!
])
    

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix

split = StratifiedShuffleSplit(n_splits=5, test_size=1/5, random_state=42)

k = 0

acc_train = list()
acc_test = list()
for train_index, test_index in split.split(X,Y):
#    print(train_index)
#    print(test_index)
    if k==0:   # print info
        print("In the training set of size{},".format(Y[train_index].shape))
        print("cor: {}, incor: {}".format(sum(Y[train_index]==1),sum(Y[train_index]==-1)))
        print("In the training set of size{},".format(Y[test_index].shape))    
        print("cor: {}, incor: {}".format(sum(Y[test_index]==1),sum(Y[test_index]==-1)))
        baseAcc = sum(Y[test_index]==1) / (sum(Y[test_index]==1)+sum(Y[test_index]==-1)) 
        print("base accuracy = {}".format(baseAcc))

    X_train = X[train_index,:]
    Y_train = Y[train_index]
    X_test = X[test_index,:]    
    Y_test = Y[test_index]
    
    clf.fit(X_train,Y_train)

    # training error
    Yhat_train = clf.predict(X_train)
    acc_train.append(np.mean(Yhat_train==Y_train))

    # test error
    Yhat_test = clf.predict(X_test)
    acc_test.append(np.mean(Yhat_test==Y_test))
 
    cm = confusion_matrix(Y_test, Yhat_test, labels=(1,-1))
    print(cm)

#     # let's manually check
#     print(sum((Y_test == 1) & (Yhat_test == 1)))    # 1s are (correctly) predicted as 1s
#     print(sum((Y_test == 1) & (Yhat_test == -1)))   # 1s are predicted as -1
#     print(sum((Y_test == -1) & (Yhat_test == 1)))   # -1s are predicted as 1
#     print(sum((Y_test == -1) & (Yhat_test == -1)))  # -1s are (correctly) predicted as -1

    
    k = k + 1
print("training accuracy")    
print(acc_train)
print("test accuracy")    
print(acc_test)
print(np.mean(acc_test))

In [ ]:
# re-train using all the data (CAN I DO THIS?)

clf.fit(X,Y)

# get coefficient of SVM
w = clf.named_steps['linear_svc'].coef_
w = w.ravel()
#w.shape
plt.plot(w)
plt.ylabel('w')
plt.xlabel('feature index')


w.shape
#listSelectedFeature.shape
listSelectedFeature[w>0.1,:]

listSelectedFeature[w<-0.1,:]


## So, conclusions are
* linear SVM does *NOT* work for p<0.05
* linear SVM does workf for p<0.1

## Nonlinear SVMs

In [ ]:
from sklearn.pipeline import Pipeline

rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", svm.SVC(kernel="rbf", gamma=2, C=10,random_state=42))   # class_weight='balanced',
    ])
    
rbf_kernel_svm_clf.fit(X, Y)

Yhat=rbf_kernel_svm_clf.predict(X)
print(np.mean(Y==Yhat))
plt.plot(Yhat)

## Cross validation with nonlinear SVM

In [ ]:
# repeat for a wide range of gamma and C

Gs=2**np.linspace(-7,3,11)
Gs
Cs = 10**np.linspace(-20,20,41)
Cs


# unbalanced 
k = 5
gs = list()
cs = list()
scores = list()
for G in Gs:
    print("gamma={}".format(G))
    for C in Cs:
        rbf_kernel_svm_clf = Pipeline([
            ("scaler", StandardScaler()),
            ("svm_clf", svm.SVC(kernel="rbf", gamma=G, C=C,random_state=42))   # class_weight='balanced',
        ])
        gs.append(G)
        cs.append(C)
        scores.append(cross_val_score(rbf_kernel_svm_clf, X, Y, cv=k))


mean_scores = np.mean(scores,axis=1)
#print(mean_scores)    


maxIdx=np.argmax(mean_scores)
print("max accuracy = {} when gamma={} and C={}".format(mean_scores[maxIdx], gs[maxIdx], cs[maxIdx]))




# save results to files
np.savetxt("gamma.txt", gs)
np.savetxt("C.txt", Cs)
np.savetxt("scores.txt", scores)
np.savetxt("mean_scores.txt", mean_scores)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(mean_scores)
#plt.plot(np.log10(Cs), mean_scores)
plt.plot(baseAcc*np.ones_like(mean_scores), '--')
plt.xlabel('(gamma,C)')
plt.ylabel('accuracy')

In [ ]:
0.8905555555555555*9

## add balance

In [ ]:
## add balance

Gs=2**np.linspace(-7,3,11)
Gs
Cs = 10**np.linspace(-20,20,41)
Cs


# unbalanced 
k = 5
gs = list()
cs = list()
scores_balanced = list()
for G in Gs:
    print("gamma={}".format(G))
    for C in Cs:
        rbf_kernel_svm_clf = Pipeline([
            ("scaler", StandardScaler()),
            ("svm_clf", svm.SVC(kernel="rbf", gamma=G, C=C, class_weight='balanced', random_state=42))   # ,
        ])
        gs.append(G)
        cs.append(C)
        scores_balanced.append(cross_val_score(rbf_kernel_svm_clf, X, Y, cv=k))


mean_scores_balanced = np.mean(scores_balanced,axis=1)
#print(mean_scores)    


maxIdx=np.argmax(mean_scores_balanced)
print("max accuracy = {} when gamma={} and C={}".format(mean_scores_balanced[maxIdx], gs[maxIdx], cs[maxIdx]))




# save results to files
np.savetxt("gamma_balanced.txt", gs)
np.savetxt("C_balanced.txt", Cs)
np.savetxt("scores_balanced.txt", scores_balanced)
np.savetxt("mean_scores_balanced.txt", mean_scores_balanced)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(mean_scores)
#plt.plot(np.log10(Cs), mean_scores)
plt.plot(baseAcc*np.ones_like(mean_scores), '--')
plt.xlabel('(gamma,C)')
plt.ylabel('accuracy')

In [ ]:
# refit using the best param
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", svm.SVC(kernel="rbf", gamma=0.03125, C=1, class_weight='balanced', random_state=42))   # ,
])

rbf_kernel_svm_clf.fit(X,Y)

# training error = 0
Yhat = rbf_kernel_svm_clf.predict(X)
np.mean(Yhat==Y)

In [ ]:
# let's repeat & check confusion matrix!

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix

split = StratifiedShuffleSplit(n_splits=5, test_size=1/5, random_state=42)

k = 0

acc_train = list()
acc_test = list()
for train_index, test_index in split.split(X,Y):
#    print(train_index)
#    print(test_index)
    if k==0:   # print info
        print("In the training set of size{},".format(Y[train_index].shape))
        print("cor: {}, incor: {}".format(sum(Y[train_index]==1),sum(Y[train_index]==-1)))
        print("In the training set of size{},".format(Y[test_index].shape))    
        print("cor: {}, incor: {}".format(sum(Y[test_index]==1),sum(Y[test_index]==-1)))
        baseAcc = sum(Y[test_index]==1) / (sum(Y[test_index]==1)+sum(Y[test_index]==-1)) 
        print("base accuracy = {}".format(baseAcc))

    X_train = X[train_index,:]
    Y_train = Y[train_index]
    X_test = X[test_index,:]    
    Y_test = Y[test_index]
    
    rbf_kernel_svm_clf.fit(X_train,Y_train)

    # training error
    Yhat_train = rbf_kernel_svm_clf.predict(X_train)
    acc_train.append(np.mean(Yhat_train==Y_train))

    # test error
    Yhat_test = rbf_kernel_svm_clf.predict(X_test)
    acc_test.append(np.mean(Yhat_test==Y_test))
 
    cm = confusion_matrix(Y_test, Yhat_test, labels=(1,-1))
    print(cm)

#     # let's manually check
#     print(sum((Y_test == 1) & (Yhat_test == 1)))    # 1s are (correctly) predicted as 1s
#     print(sum((Y_test == 1) & (Yhat_test == -1)))   # 1s are predicted as -1
#     print(sum((Y_test == -1) & (Yhat_test == 1)))   # -1s are predicted as 1
#     print(sum((Y_test == -1) & (Yhat_test == -1)))  # -1s are (correctly) predicted as -1

    
    k = k + 1
print("training accuracy")    
print(acc_train)
print("test accuracy")    
print(acc_test)
print(np.mean(acc_test))
print(np.std(acc_test))

In [ ]:
Yhat_test

In [ ]:
8/9